In [12]:
import requests
import pandas as pd
import time
import os
import random
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 基本設定（変更なし） ---
API_KEY = "90469972beed34fa2913dc1ad6a644ac" 
RESULTS_CSV_PATH = '../data/processed/citing_papers_with_paths.csv'
XML_OUTPUT_DIR = '../data/raw/fulltext/'
FULLTEXT_API_URL = "https://api.elsevier.com/content/article/doi/"
RETRY_BATCH_SIZE = 18000

# --- ダウンロード関数（変更なし）---
def download_xml_by_doi_with_retry(task, max_retries=3):
    doi = task.get('citing_paper_doi')
    if not doi or pd.isna(doi):
        task['fulltext_xml_path'] = None
        task['download_status'] = "failed (DOI is missing)"
        return task
    safe_filename = doi.replace('/', '_') + '.xml'
    xml_path = os.path.join(XML_OUTPUT_DIR, safe_filename)
    if os.path.exists(xml_path):
        task['fulltext_xml_path'] = xml_path
        task['download_status'] = "success (cached)"
        return task
    for attempt in range(max_retries):
        try:
            url = f"{FULLTEXT_API_URL}{doi}?apiKey={API_KEY}"
            response = requests.get(url, headers={'Accept': 'application/xml'}, timeout=60)
            if response.status_code == 200:
                with open(xml_path, 'w', encoding='utf-8') as f:
                    f.write(response.text)
                task['fulltext_xml_path'] = xml_path
                task['download_status'] = f"success (on retry)"
                return task
            elif response.status_code == 429:
                wait_time = (2 ** attempt) + random.uniform(0, 1)
                time.sleep(wait_time)
            else:
                task['fulltext_xml_path'] = None
                task['download_status'] = f"failed (Status: {response.status_code})"
                return task
        except requests.exceptions.RequestException:
            time.sleep((2 ** attempt) + random.uniform(0, 1))
    task['fulltext_xml_path'] = None
    task['download_status'] = f"failed (retries exhausted)"
    return task

# --- 1. 失敗したタスクの読み込み（変更なし）---
try:
    df_results = pd.read_csv(RESULTS_CSV_PATH)
    retry_targets_df = df_results[df_results['download_status'] == 'failed (Status: 429)'].copy()
    if not retry_targets_df.empty:
        batch_to_retry_df = retry_targets_df.head(RETRY_BATCH_SIZE)
        print(f"レートリミットで失敗した {len(retry_targets_df)} 件のうち、最初の {len(batch_to_retry_df)} 件の再試行を開始します。")
        tasks_to_retry = batch_to_retry_df.to_dict('records')
    else:
        print("429エラーで失敗したタスクはありませんでした。")
        tasks_to_retry = []
except FileNotFoundError:
    print(f"エラー: '{RESULTS_CSV_PATH}' が見つかりません。")
    tasks_to_retry = []

# --- 2. 失敗したタスクのみを再実行（変更なし）---
retry_results_list = []
if tasks_to_retry:
    os.makedirs(XML_OUTPUT_DIR, exist_ok=True)
    with ThreadPoolExecutor(max_workers=3) as executor:
        retry_results_list = list(tqdm(executor.map(download_xml_by_doi_with_retry, tasks_to_retry), total=len(tasks_to_retry), desc="429エラー再試行中"))

# --- 3. 元のデータと結果をマージして更新 ---
#【変更点】 .update()を使わない、より安全な方法に変更
if retry_results_list:
    # 再試行したタスクの結果をDataFrameに変換
    df_retry_results = pd.DataFrame(retry_results_list)
    
    # 再試行の対象となったDOIのリストを取得
    retried_dois = df_retry_results['citing_paper_doi'].unique()
    
    # 元のデータから、今回再試行したDOIの古い情報を除外
    df_original_without_retried = df_results[~df_results['citing_paper_doi'].isin(retried_dois)]
    
    # 除外したデータと、新しい結果を結合（concat）する
    df_updated = pd.concat([df_original_without_retried, df_retry_results], ignore_index=True)
    
    # 更新されたDataFrameを同じファイルに上書き保存
    df_updated.to_csv(RESULTS_CSV_PATH, index=False, encoding='utf-8-sig')
    
    print(f"\n再試行完了。'{RESULTS_CSV_PATH}' を更新しました。")
    print("\n--- 最新の処理結果サマリー ---")
    print(df_updated['download_status'].value_counts())
else:
    print("再試行するタスクはありませんでした。")

レートリミットで失敗した 12364 件のうち、最初の 12364 件の再試行を開始します。


429エラー再試行中:   0%|          | 0/12364 [00:00<?, ?it/s]


再試行完了。'../data/processed/citing_papers_with_paths.csv' を更新しました。

--- 最新の処理結果サマリー ---
download_status
failed (Status: 404)    95983
success (cached)        18336
success (on retry)      14798
success (downloaded)     7586
Name: count, dtype: int64
